In [13]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [1]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기

cfg = Config.fromfile('./my_config/fpn_carafe/htc_fpn-carafe_ResNest-Aug.py')


PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512) # scale지정

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512) # scale지정

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.ann_file = PREFIX + 'val.json' # validation set 평가시 사용
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512) # scale지정

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/htc_fpn-carafe_ResNest'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_24.pth')

In [15]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [17]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 4.3 task/s, elapsed: 193s, ETA:     0s

In [18]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()



class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.053750146 183.57114 366.2323 197.65417 392...,batch_01_vt/0021.jpg
1,1 0.17564188 7.117279 246.43864 305.54507 455....,batch_01_vt/0028.jpg
2,1 0.99978733 71.24503 170.73245 377.13647 377....,batch_01_vt/0031.jpg
3,1 0.10250482 293.84924 158.97354 364.50153 223...,batch_01_vt/0032.jpg
4,0 0.10631796 197.95282 274.96954 261.94183 284...,batch_01_vt/0070.jpg


# 후처리 로직

In [22]:
data = pd.read_csv('./work_dirs/submission2.csv')

submission = pd.DataFrame()

In [23]:
import os

prediction_strings = []
file_names = []

for i in range(len(data)):
    prediction_string = data.iloc[i]['PredictionString']
    file_name = data.iloc[i]['image_id']
    
    prediction_info = prediction_string.split()
    
    new_prediction_string = ""
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        if x1 != x2 and y1 != y2:
            new_prediction_string += str(class_info) + ' ' + str(prediction) + ' ' + str(x1) + ' ' + str(y1) + ' ' + str(x2) + ' ' + str(y2) + ' '
    prediction_strings.append(new_prediction_string)
    file_names.append(file_name)
    
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join('./work_dirs/', f'submission2_preprocessiong.csv'), index=None)
submission.head()

,PredictionString,image_id
0,1 0.053750146 183.57114 366.2323 197.65417 392...,batch_01_vt/0021.jpg
1,1 0.17564188 7.117279 246.43864 305.54507 455....,batch_01_vt/0028.jpg
2,1 0.99978733 71.24503 170.73245 377.13647 377....,batch_01_vt/0031.jpg
3,1 0.10250482 293.84924 158.97354 364.50153 223...,batch_01_vt/0032.jpg
4,0 0.10631796 197.95282 274.96954 261.94183 284...,batch_01_vt/0070.jpg
